In [1]:
from utils.config import *
import utils.df_processor as dfp
import utils.word_segment as ws
from IPython.display import display
from sklearn.model_selection import train_test_split

import pandas as pd
import os

### Create Dataframe

In [ ]:
df_i = pd.read_csv(I_DATA_PATH, encoding=ENCODING)
df_s = pd.read_csv(S_DATA_PATH, encoding=ENCODING)
df_i.insert(0, 'category', '內科')
df_s.insert(0, 'category', '外科')


In [ ]:
display(df_i['department'].value_counts())
display(df_s['department'].value_counts())

In [ ]:
df = pd.concat([df_i, df_s], ignore_index=True)
df.insert(0, 'cat_dep', df['category']+df['department'])
print(df.head())

### Remove Stop Words and Word Segment 
- title: 6m 30.2s
-   ask: 9m 8.3s
-   ans: 14m 32.4s

In [ ]:
df['answer_clean'] = df['answer'].apply(lambda x: ws.word_segment(x, STOP_WORDS_PATH))
df.to_csv(os.path.join(SAVING_DIR, 'df_list_ans.csv'), index=False)

In [ ]:
df['ask_clean'] = df['ask'].apply(lambda x: ws.word_segment(x, STOP_WORDS_PATH))
df.to_csv(os.path.join(SAVING_DIR, 'df_list_ask.csv'), index=False)

In [ ]:
df['title_clean'] = df['title'].apply(lambda x: ws.word_segment(x, STOP_WORDS_PATH))
df.to_csv(os.path.join(SAVING_DIR, 'df_list_title.csv'), index=False)

In [ ]:
display(df.head())

### Load CSV After WS

In [2]:
df = pd.read_csv(ALL_DATA_PATH, encoding=ENCODING)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69], got ['內科中医科' '內科乳腺科' '內科产科' '內科传染病' '內科体检' '內科健身' '內科关节科' '內科内分泌科' '內科内科'
 '內科减肥' '內科动脉导管未闭' '內科口腔科' '內科呼吸科' '內科复杂先心病' '內科外科' '內科妇产科' '內科小儿口腔科'
 '內科心外科' '內科心血管科' '內科急诊科' '內科性病科' '內科感染科' '內科手足外科' '內科整形科' '內科新生儿科'
 '內科普通内科' '內科普通外科' '內科泌尿科' '內科消化科' '內科烧伤科' '內科生殖医学科' '內科生活疾病' '內科男科'
 '內科皮肤科' '內科皮肤顽症' '內科眼科' '內科神经内科' '內科神经科' '內科神经脑外科' '內科精神心理' '內科精神疾病'
 '內科结核病' '內科美容' '內科耳鼻喉科' '內科肛肠' '內科肝病科' '內科肝胆科' '內科肾内科' '內科肿瘤科' '內科胸外科'
 '內科脊柱外科' '內科营养保健科' '內科血液科' '內科血管科' '內科计划生育' '內科颌面外科' '內科风湿免疫科' '內科骨科'
 '外科乳腺科' '外科呼吸科' '外科心外科' '外科普通外科' '外科泌尿科' '外科神经脑外科' '外科肛肠' '外科肝病科' '外科肝胆科'
 '外科肿瘤科' '外科胸外科' '外科血管科']

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], got ['內科内分泌科' '內科呼吸科' '內科心血管科' '內科感染科' '內科普通内科' '內科消化科' '內科神经科' '內科肝病科'
 '內科肾内科' '內科血液科' '內科风湿免疫科' '外科乳腺科' '外科心外科' '外科普通外科' '外科泌尿科' '外科神经脑外科'
 '外科肛肠' '外科肝胆科' '外科胸外科' '外科血管科']

In [3]:
idx_lst= [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
dep_lst= ['內科内分泌科', '內科呼吸科', '內科心血管科', '內科感染科', '內科普通内科', '內科消化科', '內科神经科', '內科肝病科'
, '內科肾内科', '內科血液科', '內科风湿免疫科', '外科乳腺科', '外科心外科', '外科普通外科', '外科泌尿科', '外科神经脑外科'
, '外科肛肠', '外科肝胆科', '外科胸外科', '外科血管科']

df.insert(1, 'dep_id', df['cat_dep'].apply(lambda x: dep_lst.index(x) if x in dep_lst else 99))

In [3]:
display(df.head())

,cat_dep,dep_id,category,department,title,ask,answer,answer_clean,ask_clean,title_clean
0,內科心血管科,2,內科,心血管科,高血压患者能吃党参吗？,我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？,高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病...,"['高血压', '病人', '口服', '党参', '党参', '降血脂', '降血压', ...","['高血压', '两天', '女婿', '些', '党参', '泡水', '喝', '您好'...","['高血压', '患者', '吃', '党参']"
1,內科心血管科,2,內科,心血管科,高血压该治疗什么？,我是一位中学教师，平时身体健康，最近学校组织健康检查，结果发觉我是高血压，去年还没有这种情况...,高血压患者首先要注意控制食盐摄入量，每天不超过六克，注意不要吃太油腻的食物，多吃新鲜的绿色蔬...,"['高血压', '患者', '控制', '食盐', '摄入量', '不', '超过', '六...","['一位', '中学教师', '平时', '身体健康', '学校', '组织', '健康检查...","['高血压', '治疗']"
2,內科心血管科,2,內科,心血管科,老年人高血压一般如何治疗？,我爷爷今年68了，年纪大了，高血压这些也领着来了，这些病让老人很痛苦，每次都要按时喝药，才能...,你爷爷患高血压，这是老年人常见的心血管病，血管老化硬化，血压调整能力消退了，目前治疗高血压最...,"['爷爷', '患', '高血压', '这是', '老年人', '常见', '心血管病', ...","['爷爷', '68', '年纪', '大', '高血压', '领着', '病', '老人'...","['老年人', '高血压', '治疗']"
3,內科内分泌科,0,內科,内分泌科,糖尿病还会进行遗传吗？,糖尿病有隔代遗传吗？我妈是糖尿病，很多年了，也没养好，我现在也是，我妹子也是，我儿子现在二十...,2型糖尿病的隔代遗传概率为父母患糖尿病，临产的发生率为40%，比一般人患糖尿病，疾病，如何更...,"['型', '糖尿病', '隔代遗传', '概率', '父母', '患', '糖尿病', '...","['糖尿病', '隔代遗传', '我妈', '糖尿病', '很多年', '没养', '好',...","['糖尿病', '还会', '遗传']"
4,內科内分泌科,0,內科,内分泌科,糖尿病一般需要怎么治疗？,我妈定期检查仔细检查的时候，仔细检查出患糖尿病，糖尿病需要有怎么治疗？我大概知晓是需要有控制...,糖尿病患者首先通过饮食控制和锻练运动，肥胖患者把体重降下来等方式调整一下看一看，如果血糖仍然...,"['糖尿病', '患者', '饮食', '控制', '锻练', '运动', '肥胖', '患...","['我妈', '定期检查', '仔细检查', '仔细检查', '出患', '糖尿病', '糖...","['糖尿病', '治疗']"


In [5]:
df.to_csv(os.path.join(SAVING_DIR, 'df_id.csv'), index=False)

In [3]:
df= dfp.df_filt(df, DEP_MIN_AMOUNT)
display(dfp.get_dep_counts(df))
print(len(dfp.get_dep_counts(df)))

內科神经科      46844
內科消化科      32245
內科呼吸科      27931
外科肛肠       24016
外科神经脑外科    23620
內科心血管科     22841
內科内分泌科     21745
外科普通外科     21179
內科肝病科      20888
外科泌尿科      18422
內科肾内科      14010
內科普通内科     13447
內科血液科       9968
外科肝胆科       8831
外科乳腺科       8823
外科血管科       6404
內科风湿免疫科     5486
內科感染科       4035
外科胸外科       2913
外科心外科       1777
Name: cat_dep, dtype: int64

20


In [ ]:
display(df.head())

In [ ]:
# df['ask_clean']= df['ask_clean'].apply(lambda x: x.split(' '))

In [ ]:
df = pd.read_csv(ALL_DATA_PATH, encoding=ENCODING)


In [ ]:
df.insert(6, 'title_clean', df['title'].apply(lambda x: ws.word_segment(x)))
df.to_csv(os.path.join(SAVING_DIR, 'df_3.csv'), index=False)

In [ ]:
display(df.head())

### Split

In [4]:
x_train, x_test, y_train, y_test= train_test_split(df['ask_clean'], df['dep_id'], test_size=0.2, shuffle=True)

In [5]:
from ast import literal_eval
x_train= [literal_eval(lst) for lst in x_train]
x_test= [literal_eval(lst) for lst in x_test]

In [ ]:
print(type(x_train), type(x_train[0]))

In [6]:
idx= 0
display(x_train[idx], y_train[idx])

['患者',
 '性别',
 '女患者',
 '年龄',
 '60',
 '肝硬化',
 '脾脏',
 '切除',
 '手术',
 '一个多月',
 '后',
 '发烧',
 '现象',
 '何原因',
 '服用',
 '药']

2

In [ ]:
display(x_train.head())
display(y_train.head())

### From Ref

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
 
tfidf = TfidfVectorizer(analyzer=lambda x: x)
logistic = LogisticRegression(solver='liblinear', multi_class='auto')

tfidf_logistic = Pipeline([
    ('tfidf', tfidf), 
    ('logistic', logistic)
])

In [ ]:
from scipy.stats import randint, uniform

# w2v_params = {'w2v__size': [100, 150, 200]}
tfidf_params = {'tfidf__ngram_range': [(1, 1), (1, 2)]}
logistic_params = {'logistic__C': [0.5, 1.0, 1.5]}
# xgb_params = {'xgb__max_depth': randint(low=3, high=12),
#               'xgb__colsample_bytree': uniform(loc=0.8, scale=0.2),
#               'xgb__subsample': uniform(loc=0.8, scale=0.2)}

tfidf_logistic_params = {**tfidf_params, **logistic_params}
# w2v_xgb_params = {**w2v_params, **xgb_params}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

cv = 3
n_iter = 3
random_state = 1234
scoring = 'accuracy'

all_models = [
    ('tfidf_logistic', tfidf_logistic, tfidf_logistic_params)
]

all_models_info = []
for name, model, params in all_models:
    print('training:', name)
    model_tuned = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        cv=cv,
        n_iter=n_iter,
        n_jobs=-1,
        verbose=1,
        scoring=scoring,
        random_state=random_state,
        return_train_score=False
    ).fit(x_train, y_train)
    
    Y_test_pred = model_tuned.predict(x_test)
    test_score = accuracy_score(y_test, Y_test_pred)
    info = name, model_tuned.best_score_, test_score, model_tuned
    all_models_info.append(info)

columns = ['model_name', 'train_score', 'test_score', 'estimator']
results = pd.DataFrame(all_models_info, columns=columns)
results = (results
           .sort_values('test_score', ascending=False)
           .reset_index(drop=True))
results

### G

In [7]:
# from sklearn_api.w2vmodel import W2VTransformer
from sklearn_api.gensim_word2vec import GensimWord2VecVectorizer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

xgb_model= Pipeline([
    # ('w2v', W2VTransformer(size=100, min_count=3, workers=8, window=7)),
    ('w2v', GensimWord2VecVectorizer(size=100, min_count=3, workers=8, window=7)),
    ('xgb', XGBClassifier(
        learning_rate=0.01, 
        objective='multi:softmax', 
        eval_metric='mlogloss', 
        scale_pos_weight=1, 
        colsample_btree=0.8,
        subsample=0.9))
])

In [8]:
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform
import matplotlib.pyplot  as plt
from sklearn.metrics import accuracy_score

xgb_model.fit(x_train, y_train)

y_test_pred = xgb_model.predict(x_test)
test_score = accuracy_score(y_test, y_test_pred)


### model evaluate
accuracy = accuracy_score(y_test,y_test_pred)
print("accuarcy: %.5f%%" % (accuracy*100.0))

[19:09:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_btree", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [ ]:
import joblib
import os
joblib.dump(xgb_model, os.path.join(SAVING_DIR, 'xgb_model.joblib'))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint, uniform
from sklearn_api.w2vmodel import W2VTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
# , window=7
xgb_model= Pipeline([
    ('w2v', W2VTransformer(size=100, min_count=3, workers=8)),
    ('xgb', XGBClassifier(
        learning_rate=0.01, 
        objective='multi:softmax', 
        eval_metric='mlogloss', 
        scale_pos_weight=1))
])
cv = 3
n_iter = 3
random_state = 9527
scoring = 'accuracy'

w2v_params = {'w2v__window': randint(low=5, high=10)}

xgb_params = {'xgb__max_depth': randint(low=3, high=12),
              'xgb__colsample_bytree': uniform(loc=0.8, scale=0.2),
              'xgb__subsample': uniform(loc=0.8, scale=0.2)}

params={**w2v_params, **xgb_params}


grid = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=params,
    cv=cv,
    n_iter=n_iter,
    n_jobs=-1,
    verbose=1,
    scoring=scoring,
    random_state=random_state,
    return_train_score=False
).fit(x_train, y_train)

y_test_pred = grid.predict(x_test)
test_score = accuracy_score(y_test, y_test_pred)
info = name, grid.best_score_, test_score, grid
all_models_info.append(info)
best_estimator = grid.best_estimator_
print('best:', best_estimator, grid.best_score_)

columns = ['model_name', 'train_score', 'test_score', 'estimator']
results = pd.DataFrame(all_models_info, columns=columns)
results = (results
           .sort_values('test_score', ascending=False)
           .reset_index(drop=True))
results

In [ ]:
display(df_all.head())

### Saving Text&CSV

In [ ]:
# df_p.i_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['ask'], index=False, header=False)
# df_p.i_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['answer'], index=False, header=False)
# df_p.s_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['ask'], index=False, header=False)
# df_p.s_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['answer'], index=False, header=False)

In [ ]:
df_p.all_df['answer_clean'] = df_p.all_df['answer'].apply(
    lambda x: ws.word_segment(x))

In [ ]:
display(df_p.all_df.head())

### Main Method